<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/chooser_option.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 86.6 MB/s eta 0:00:00


In [3]:
import QuantLib as ql
import math

print(f"QuantLib Version: {ql.__version__}")

# Helper function for reporting errors (similar to C++ REPORT_FAILURE)
def report_failure(test_name, value_name, params, expected, calculated, tolerance):
    print(f"--- FAILURE IN {test_name} ---")
    print(f"Calculating: {value_name}")
    print("Parameters:")
    for key, value in params.items():
        print(f"  {key}: {value}")
    print(f"  Expected:   {expected:.5f}")
    print(f"  Calculated: {calculated:.5f}")
    print(f"  Difference: {abs(calculated - expected):.5f}")
    print(f"  Tolerance:  {tolerance:.5f}")
    print("-" * 30)
    return False # Indicate failure

def check_result(test_name, value_name, params, expected, calculated, tolerance):
    if abs(calculated - expected) > tolerance:
        return report_failure(test_name, value_name, params, expected, calculated, tolerance)
    else:
        print(f"{test_name}: {value_name} check PASSED")
        print(f"  Expected: {expected:.5f}, Calculated: {calculated:.5f}, Diff: {abs(calculated-expected):.5f} <= Tol: {tolerance:.5f}")
        return True # Indicate success

QuantLib Version: 1.38


In [4]:
print("\nTesting analytic simple chooser option...")

# --- Setup ---
day_count = ql.Actual360()
# Use a fixed date for reproducibility, mirroring test environment
eval_date = ql.Date(15, 5, 2023) # Example date
ql.Settings.instance().evaluationDate = eval_date
calendar = ql.TARGET() # Needed for vol surface

# --- Market Data (matching C++ example) ---
spot_value = 50.0
q_rate_value = 0.0
r_rate_value = 0.08
vol_value = 0.25

spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_value))
q_ts_handle = ql.YieldTermStructureHandle(ql.FlatForward(eval_date, q_rate_value, day_count))
r_ts_handle = ql.YieldTermStructureHandle(ql.FlatForward(eval_date, r_rate_value, day_count))
vol_ts_handle = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(eval_date, calendar, vol_value, day_count))

# --- Stochastic Process ---
process = ql.BlackScholesMertonProcess(spot_handle, q_ts_handle, r_ts_handle, vol_ts_handle)

# --- Pricing Engine ---
engine = ql.AnalyticSimpleChooserEngine(process)

# --- Option Definition ---
strike = 50.0
exercise_days = 180
choosing_days = 90

exercise_date = eval_date + ql.Period(exercise_days, ql.Days)
choosing_date = eval_date + ql.Period(choosing_days, ql.Days)

exercise = ql.EuropeanExercise(exercise_date)
option = ql.SimpleChooserOption(choosing_date, strike, exercise)
option.setPricingEngine(engine)

# --- Calculation and Check ---
calculated_npv = option.NPV()
expected_npv = 6.1071
tolerance = 3e-5

# Prepare parameters for reporting
params = {
    "Spot": spot_value,
    "Dividend Rate": q_rate_value,
    "Risk-Free Rate": r_rate_value,
    "Volatility": vol_value,
    "Strike": strike,
    "Choosing Date": choosing_date,
    "Exercise Date": exercise_date,
    "Evaluation Date": eval_date
}

simple_test_passed = check_result(
    "AnalyticSimpleChooser", "NPV", params,
    expected_npv, calculated_npv, tolerance
)

if not simple_test_passed:
    # Optional: raise an error to mimic BOOST_ERROR behavior
    # raise AssertionError("Simple chooser test failed!")
    pass


Testing analytic simple chooser option...
AnalyticSimpleChooser: NPV check PASSED
  Expected: 6.10710, Calculated: 6.10708, Diff: 0.00002 <= Tol: 0.00003


In [5]:
print("\nTesting analytic complex chooser option...")

# --- Setup ---
day_count = ql.Actual360()
# Use a fixed date, can be different from previous test if needed
eval_date = ql.Date(16, 5, 2023) # Example date
ql.Settings.instance().evaluationDate = eval_date
calendar = ql.TARGET()

# --- Market Data (matching C++ example) ---
spot_value = 50.0
q_rate_value = 0.05
r_rate_value = 0.10
vol_value = 0.35

spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_value))
q_ts_handle = ql.YieldTermStructureHandle(ql.FlatForward(eval_date, q_rate_value, day_count))
r_ts_handle = ql.YieldTermStructureHandle(ql.FlatForward(eval_date, r_rate_value, day_count))
vol_ts_handle = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(eval_date, calendar, vol_value, day_count))

# --- Stochastic Process ---
process = ql.BlackScholesMertonProcess(spot_handle, q_ts_handle, r_ts_handle, vol_ts_handle)

# --- Pricing Engine ---
engine = ql.AnalyticComplexChooserEngine(process)

# --- Option Definition ---
call_strike = 55.0
put_strike = 48.0
choosing_days = 90
call_exercise_offset_days = 180 # Days *after* choosing date
put_exercise_offset_days = 210 # Days *after* choosing date

choosing_date = eval_date + ql.Period(choosing_days, ql.Days)
call_exercise_date = choosing_date + ql.Period(call_exercise_offset_days, ql.Days)
put_exercise_date = choosing_date + ql.Period(put_exercise_offset_days, ql.Days)

call_exercise = ql.EuropeanExercise(call_exercise_date)
put_exercise = ql.EuropeanExercise(put_exercise_date)

option = ql.ComplexChooserOption(
    choosing_date,
    call_strike,
    put_strike,
    call_exercise,
    put_exercise
)
option.setPricingEngine(engine)

# --- Calculation and Check ---
calculated_npv = option.NPV()
expected_npv = 6.0508
tolerance = 1e-4

# Prepare parameters for reporting
params = {
    "Spot": spot_value,
    "Dividend Rate": q_rate_value,
    "Risk-Free Rate": r_rate_value,
    "Volatility": vol_value,
    "Call Strike": call_strike,
    "Put Strike": put_strike,
    "Choosing Date": choosing_date,
    "Call Exercise Date": call_exercise_date,
    "Put Exercise Date": put_exercise_date,
    "Evaluation Date": eval_date
}

complex_test_passed = check_result(
    "AnalyticComplexChooser", "NPV", params,
    expected_npv, calculated_npv, tolerance
)

if not complex_test_passed:
    # Optional: raise an error
    # raise AssertionError("Complex chooser test failed!")
    pass

print("\nChooser option tests finished.")


Testing analytic complex chooser option...
AnalyticComplexChooser: NPV check PASSED
  Expected: 6.05080, Calculated: 6.05079, Diff: 0.00001 <= Tol: 0.00010

Chooser option tests finished.
